In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
sys.path.insert(0, os.path.abspath("../"))

from functions_extra.metrics import Relations

In [111]:
dados_brutos = "../data_sampling/dados_brutos/%s"
dados_tratados = "../data_sampling/dados_tratados/%s"

In [112]:
df = pd.read_parquet(dados_tratados%"df_cat.parquet")

In [113]:
vars_cat = df.columns[1:-2]

In [114]:
rel = Relations(df, resp='resp')

In [115]:
cramer_cor = rel.get_cramers_V(vars_cat, ["resp"])

In [128]:
cramer_cor.sort_values(by="resp", ascending=False).reset_index()

,index,resp
0,Contract,0.410116
1,tenure,0.372885
2,OnlineSecurity,0.347400
3,TechSupport,0.342916
4,InternetService,0.322455
5,PaymentMethod,0.303359
6,OnlineBackup,0.292316
7,DeviceProtection,0.281580
8,TotalCharges,0.281424
9,StreamingMovies,0.230951


In [117]:
map_woe, map_iv, map_count, map_fraud = rel.get_woe(vars_cat, target="resp")

In [118]:
for k,v in map_woe.items():
    display(k)
    display(
        pd.DataFrame({
            "category": map_woe[k].keys(), 
            "n": [int(x[0]) for x in map_count[k].values()],
            "churn": [int(x[0]) for x in map_fraud[k].values()],
            "woe": map_woe[k].values(),
            })\
                .sort_values("woe", ascending=False)
                .style.bar(
                    subset=["woe"], color='#d65f5f'))

'gender'

,category,n,churn,woe
0,Female,3488,939,0.019767
1,Male,3555,930,-0.019233


'SeniorCitizen'

,category,n,churn,woe
1,1,1142,476,0.682499
0,0,5901,1393,-0.156074


'Partner'

...

,category,n,churn,woe
1,Electronic check,2365,1071,0.829007
2,Mailed check,1612,308,-0.423782
0,Bank transfer (automatic),1544,258,-0.586713
3,Credit card (automatic),1522,232,-0.695823


'MonthlyCharges'

,category,n,churn,woe
2,"(74.125, 86.325]",1250,487,0.569449
1,"(86.325, 100.275]",1158,433,0.503102
0,"(100.275, inf]",868,236,0.034340
4,"(49.425, 55.975]",492,117,-0.143748
3,"(55.975, 74.125]",1041,247,-0.148231
5,"(20.875, 49.425]",1093,252,-0.185703
7,"(-inf, 19.925]",569,52,-1.270125
6,"(19.925, 20.875]",572,45,-1.432365


'TotalCharges'

,category,n,churn,woe
7,"(58.575, 96.625]",362,244,1.742374
8,"(-inf, 58.575]",447,183,0.652445
6,"(96.625, 365.725]",882,318,0.445754
5,"(365.725, 1240.525]",1592,428,0.018315
2,"(2108.85, 4026.0]",1142,273,-0.138545
3,"(4026.0, 4838.375]",397,75,-0.433899
4,"(1240.525, 2108.85]",989,181,-0.475853
1,"(4838.375, 6976.0]",891,131,-0.736898
0,"(6976.0, inf]",341,36,-1.106566


In [119]:
recat = {
    "tenure": {
        "(-inf, 1.5]":"(-inf, 10.5]",
        "(1.5, 10.5]":"(-inf, 10.5]",
        "(22.5, 33.5]":"(22.5, 59.5]",
        "(33.5, 45.5]":"(22.5, 59.5]",
        "(45.5, 59.5]":"(22.5, 59.5]",
        "(59.5, 64.5]":"(64.5.5, inf]",
        "(64.5, inf]":"(64.5.5, inf]"
    },

    "PaymentMethod" : {
        "Bank transfer (automatic)":"Automatic payment",
        "Credit card (automatic)":"Automatic payment" 	

    },

    "MonthlyCharges": {
        "(-inf, 19.925]":"(-inf, 20.875]",
        "(19.925, 20.875]":"(-inf, 20.875]",
        "(20.875, 49.425]":"(20.875, 74.125]",
        "(49.425, 55.975]":"(20.875, 74.125]",
        "(55.975, 74.125]":"(20.875, 74.125]",
        "(74.125, 86.325]":"(74.125, inf]",
        "(86.325, 100.275]":"(74.125, inf]",
        "(100.275, inf]": "(74.125, inf]"
    },

    "TotalCharges": {
        "(365.725, 1240.525]":"(365.725, 6976.0]",
        "(1240.525, 2108.85]":"(365.725, 6976.0]",
        "(2108.85, 4026.0]":"(365.725, 6976.0]",
        "(4026.0, 4838.375]": "(365.725, 6976.0]",
        "(4838.375, 6976.0]":"(365.725, 6976.0]"
    },

    "MultipleLines": {"No phone service": "No"}

}

In [120]:
df = df.replace(recat)

In [121]:
rel = Relations(df, resp='resp')

In [122]:
map_woe, map_iv, map_count, map_fraud = rel.get_woe(vars_cat, target="resp")

In [123]:
for k,v in map_woe.items():
    display(k)
    display(
        pd.DataFrame({
            "category": map_woe[k].keys(), 
            "n": [int(x[0]) for x in map_count[k].values()],
            "churn": [int(x[0]) for x in map_fraud[k].values()],
            "woe": map_woe[k].values(),
            })\
                .sort_values("woe", ascending=False)
                .style.bar(
                    subset=["woe"], color='#d65f5f'))

'gender'

,category,n,churn,woe
0,Female,3488,939,0.019767
1,Male,3555,930,-0.019233


'SeniorCitizen'

,category,n,churn,woe
1,1,1142,476,0.682499
0,0,5901,1393,-0.156074


'Partner'

...

,category,n,churn,woe
1,Electronic check,2365,1071,0.829007
2,Mailed check,1612,308,-0.423782
0,Automatic payment,3066,490,-0.640690


'MonthlyCharges'

,category,n,churn,woe
0,"(74.125, inf]",3276,1156,0.411818
1,"(20.875, 74.125]",2626,616,-0.164009
2,"(-inf, 20.875]",1141,97,-1.353370


'TotalCharges'

,category,n,churn,woe
3,"(58.575, 96.625]",362,244,1.742374
4,"(-inf, 58.575]",447,183,0.652445
2,"(96.625, 365.725]",882,318,0.445754
1,"(365.725, 6976.0]",5011,1088,-0.264112
0,"(6976.0, inf]",341,36,-1.106566


In [124]:
pd.DataFrame({"var": map_iv.keys(), "IV": map_iv.values()}).sort_values(by="IV", ascending=False)

,var,IV
14,Contract,1.234536
4,tenure,0.735656
8,OnlineSecurity,0.716313
11,TechSupport,0.698135
7,InternetService,0.616694
9,OnlineBackup,0.527459
10,DeviceProtection,0.498575
16,PaymentMethod,0.455573
13,StreamingMovies,0.380339
12,StreamingTV,0.379428


In [125]:
df = df[vars_cat].replace(map_woe).copy()

In [126]:
df.to_parquet(dados_tratados%"df_woe.parquet")

In [127]:
import pickle
with open('../notebooks/artefatos/map_woe.pickle', 'wb') as handle:
    pickle.dump(map_woe, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../notebooks/artefatos/vars_cat.pickle', 'wb') as handle:
    pickle.dump(vars_cat, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../notebooks/artefatos/recat.pickle', 'wb') as handle:
    pickle.dump(recat, handle, protocol=pickle.HIGHEST_PROTOCOL)